In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime,timedelta
from tqdm import tqdm
import pandas as pd 

In [2]:
cookies = {
    'twk_uuid_5785e4dff205e88262fd49f0': '%7B%22uuid%22%3A%221.2U63Ftif6AyHpK2T22E9zvOZQ4GMgFl9aHi11hYkh7EvkEG5In7Z2zTAh02GLna0m93mtylfH87OEMQYkkn3gySNLFuwIwNwpFsXtWSQbk8zRjo7oqg03zAOX5txA2f%22%2C%22version%22%3A3%2C%22domain%22%3A%22selgebeachhotel.com%22%2C%22ts%22%3A1707738072347%7D',
    'ASP.NET_SessionId': 'vzuvj2zbg4j14g23vuao34ad',
    'Cerez': 'languagePref=tr-TR&LangID=1&LangCode=TR',
    'CerezOpt': 'OperatorCode=WEBTL&Ulke=TR',
    'OtelAra': 'CIn=08.06.2024&COut=09.06.2024&Adl=2&Chd=0&ChdAge=',
}

headers = {
    'authority': 'online.selgebeachhotel.com',
    'accept': '*/*',
    'accept-language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'cookie': 'twk_uuid_5785e4dff205e88262fd49f0=%7B%22uuid%22%3A%221.2U63Ftif6AyHpK2T22E9zvOZQ4GMgFl9aHi11hYkh7EvkEG5In7Z2zTAh02GLna0m93mtylfH87OEMQYkkn3gySNLFuwIwNwpFsXtWSQbk8zRjo7oqg03zAOX5txA2f%22%2C%22version%22%3A3%2C%22domain%22%3A%22selgebeachhotel.com%22%2C%22ts%22%3A1707738072347%7D; ASP.NET_SessionId=vzuvj2zbg4j14g23vuao34ad; Cerez=languagePref=tr-TR&LangID=1&LangCode=TR; CerezOpt=OperatorCode=WEBTL&Ulke=TR; OtelAra=CIn=08.06.2024&COut=09.06.2024&Adl=2&Chd=0&ChdAge=',
    'origin': 'https://online.selgebeachhotel.com',
    'pragma': 'no-cache',
    'referer': 'https://online.selgebeachhotel.com/?strInDate=08.06.2024&strOutDate=09.06.2024&intAdl=2&intChd=0',
    'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

data = {
    'sender': 'btnOdaAra',
    'txtGirisTarihi': [
        '08.03.2024',
        '09.03.2024',
    ],
    'ddAdl': '2',
    'ddChd': '0',
    'form': 'otelara',
    'formname': 'formOda',
    'hotelcode': 'SELGE',
    'operatorcode': 'WEBTL',
}

response = requests.post('https://online.selgebeachhotel.com/Ajx.aspx', cookies=cookies, headers=headers, data=data)

In [3]:
def make_request(data):

    response = requests.post('https://online.selgebeachhotel.com/Ajx.aspx', cookies=cookies, headers=headers, data=data)

    return response

In [4]:
def fetch_data(check_in_date):

    check_out_date = check_in_date + timedelta(days=1)
    
    check_in_date = check_in_date.strftime('%d.%m.%Y')
    check_out_date = check_out_date.strftime('%d.%m.%Y')

    data['txtGirisTarihi'] = [check_in_date,check_out_date]

    response = make_request(data)
    
    soup = BeautifulSoup(response.content,'lxml')

    rooms = soup.find_all('div', class_='row')

    # Initialize a list to hold room data
    room_data = []

    for room in rooms:
        room_name = room.find('h3').text if room.find('h3') else 'No Name'
        room_price = room.find('span', class_='oda-yeni-fiyat').text if room.find('span', class_='oda-yeni-fiyat') else 'No Price'
        amenities_div = soup.find('div', class_='tj')
        amenities_text = amenities_div.p.text.strip() if amenities_div and amenities_div.p else 'No amenities listed'
        room_data.append({'check_in_date': check_in_date, 'check_out_date': check_out_date, 'room_name': room_name, 'price': room_price, 'amenities': amenities_text})
    
    if len(room_data) == 0:
        print(f'No rooms available for {check_in_date}')
        return None

    df = pd.DataFrame(room_data)
    
    return df


In [5]:
initial_date = datetime(year=2024, month=4, day=9)
last_date = datetime(year=2025, month=1, day=1)

In [6]:
from datetime import datetime, timedelta
def datetime_range(start, end, delta=timedelta(days=1)):
    current = start
    while current < end:
        yield current
        current += delta

all_dates = list(datetime_range(initial_date, last_date))


In [7]:
df_list = []

for check_in_date in tqdm(all_dates):

    response = fetch_data(check_in_date)

    if response is not None:

        df_list.append(response)

    

  0%|          | 0/267 [00:00<?, ?it/s]

 78%|███████▊  | 207/267 [03:00<00:48,  1.25it/s]

No rooms available for 01.11.2024


 78%|███████▊  | 208/267 [03:00<00:37,  1.58it/s]

No rooms available for 02.11.2024


...

No rooms available for 29.12.2024


100%|█████████▉| 266/267 [03:10<00:00,  4.55it/s]

No rooms available for 30.12.2024


100%|██████████| 267/267 [03:11<00:00,  1.40it/s]

No rooms available for 31.12.2024


In [8]:
df = pd.concat(df_list)

In [11]:
df.drop_duplicates(inplace=True)

In [14]:
df = df[df['room_name'] != 'No Name']

In [15]:
df.head()

,check_in_date,check_out_date,room_name,price,amenities
1,09.04.2024,10.04.2024,Standart Oda,"7900,00 TL","\n\tTemizlik hizmeti, uyandırma servisi, balko..."
3,09.04.2024,10.04.2024,Standart Deniz Manzaralı,"8400,00 TL","\n\tTemizlik hizmeti, uyandırma servisi, balko..."
5,09.04.2024,10.04.2024,Deluxe Oda,"8900,00 TL","\n\tTemizlik hizmeti, uyandırma servisi, balko..."
7,09.04.2024,10.04.2024,Deluxe Oda Deniz Manzara,"9500,00 TL","\n\tTemizlik hizmeti, uyandırma servisi, balko..."
9,09.04.2024,10.04.2024,SUIT,"10000,00 TL","\n\tTemizlik hizmeti, uyandırma servisi, balko..."


In [16]:
df.to_excel('selge_beach_hotel.xlsx', index=False)